In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy.stats as ss

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config Completer.use_jedi = False
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

/var/folders/xp/ytd5822d6t3dv32kzyb93zm40000gq/T/ipykernel_87984/2666219287.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


<h1> Что у меня за распределени? </h1>

В  Т-Банке мы прогнозируем нагрузку на колл-центр (сколько придет обращений, какой длительности и некоторые другие характеристики). 
И нужно уметь переводить эту нагрузку в количество людей на линии. Для чего мы реализовали в том числе симулятор колл-центра. А для его работы нужно понимать, как распределены те или иные случайные величины. И иметь численные характеристики соответствия наблюдаемых значений теоретическим распределениям.

<b> Это называется задачей (критериями) согласия: к какому теоретическому распределению вероятностей принадлежит данная выборка. </b>

Используй Колмогорова-Смирнова - да и все тут - скажут мне. В принципе да, но нет. Мы пойдем чуть дальше и попытаемся разобраться, как оно устроено и какие у этого есть ограничения.

Из статьи вы узнаете - почему нельзя просто так применять критерии согласия, к каким это приводит ошибкам и как с этим быть.

В [прошлой статье](https://habr.com/ru/articles/815291/) мы концентрировались на сравнении средних с помощью разных критеривев (это задача <b>однородности</b> - принадлежат ли две выборки одному распределению). Для их сравнения используется p-value (надеюсь ты знаешь что это такое, если нет - то смотри [на вики](https://ru.wikipedia.org/wiki/P-%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D0%B5) и об этом есть ремарка в предыдущей статье). Для определния к какому классу относится наше распределение хотелось бы иметь что-то подобное.

Начнем давайте с обзора литературы на хабре. Чтобы лишнего не плодить сущности. Кому не интересно - листай дальше.
<details>
    <summary>Под кат</summary>
На Хабре есть ряд статей про распределения. [Вот тут](https://habr.com/ru/companies/skillfactory/articles/674880/) рассказывается про то как сравнивать распределения в принципе (статья обширная но есть явные ошибки например "Альтернативный тест — U-критерий Манна — Уитни, который сравнивает медиану двух распределений"). Там же упоминается Критерий Колмогорова-Смирнова и критерий Хи2. [Тут](https://habr.com/ru/articles/575116/) автор пытается отличить 
экспоненциальное от степенного. Интересная статья тут про [кривые Пирсона](https://habr.com/ru/companies/aktiv-company/articles/823510/) .

[Тут](https://habr.com/ru/articles/311092/) и [тут](https://habr.com/ru/articles/331060/) в принципе обзор основных распределений 
</details>

Но общей картины - зачем так много разных тестов - из них не складывается.

Начнем с того, что гипотезы могут быть простыми и сложными. Например простая гипотеза - данная выборка принадлжит стандартному нормальному распределению (т.е.
нормальному распределению с матожиданием 0 и дисперсией 1). Сложная гипотеза - данное распределение принадлежит нормальному распределению с _неизвестными_
параметрами. Эти параметры еще нужно каким-то образом определить. Причем всегда -  с конечной точностью, что увеличивает ошибку.


Второе, о чем следует упомянуть - это <b> функция мощности критерия </b>. Она показывает вероятность совершить ошибку первого рода - отвергнуть нулевую гипотизу когда она верна. Например есть два критерия (функции, которым скармливается выборка). Мы скармливаем им выбоки из нормального распределения. Более мощный критерий будет РЕЖЕ говорить - что наша выборка НЕ ИЗ нормального распределения, когда она ИЗ НОРМАЛНОГО.

"Классический" подход - это криетрии Колмогорова-Смирнова и хи2.

Мы их сейчас вкратце опишем, а потом сравним с методом Монте-Карло.

<h2> Колмогоров-Смирнов. </h2>

Он для непрерывных и обычно одномерных распределений.

Он сравнивает эмпирическую и теоретическую функции распределения. И выглядит так:

$$ D_n = sup|F_n(x) - F(x)| $$

$F_n(x)$ - это эмперическая функция распределения, $F(x)$ - теоретическая.

Т.е. мы берем две функцию распределения и находим их максимальное различие. Пусть не смущает, что мы тут ищем только одну точку (а не какое-то среднее). Функция распределения - кумулятивная - это и есть уже нечто усредненное. Чтобы получить различие - нужно чтобы для каждого значения различия накапливались.

В scipy тест реализован функцией stats.ks_1samp

<details>
    <summary>Под кат</summary>
Супремум - это, к сожалению, не максимум. Например отрезок [0,1) не имеет максимума, но супремум у него 1. Вот и тут вычисляются верхние и нижнии границы - см. ниже как. Лично для меня это не супер очевидный момент.  
</details>

Как и в случае со сравнением средних - максимальное различие функций распределения - тоже случайная величина. У которой есть своя функция плотности вероятности. Ее мы и используем для определения p-value. 

Как мы помним из предыдущей статьи - t-распределение возникает у нас в случае если дисперсия генернальной совокупности нам неизвестна. И ее мы вынуждены определять по самой экспериментальной выборке. Знай мы ее, t-распределение просто превратилось бы в Гауссово и не имело бы такие тяжелые хвосты. Собственно в гауссово оно и превращается по мере увеличения выборки и "уточнения" истинной дисперсии. 

Так же и тут. Как мы помним, сложная гипотеза это та, в которой параметры распределения нужно определять по выборке. Но вот беда - аналитического решения (т.е. "своего t-распределения") для него в общем виде нет.  

Т.е. если мы хотим ответить на вопрос: "является ли эта выборка семплированной из Гамма распределения с параметрами $\alpha = \alpha_0, \beta = \beta_0 $", то значения статистики, вообще говоря, будут зависит как от размера выборки, так и от _значения_ его параметров.

Чтобы немного понять масштаб проблемы - представьте что вы сравниваете
два средних. И вот ваше p-value в каком-нибудь t-тесте зависит от того, сравниваете ли вы 3 и 4 или 30 и 40. Не говоря уж про размер выборки. В t-тесте он идет как количество степеней свободы, а тут так не выходит.

Все это приводит к тому, что по хорошему нужно проводить моделирования мощности критерия для каждого значения параметров, для каждого распределения и еще для каждого способа нахождения параметров распределения по выборке (хотя опять же, для ряда распределений есть точные решения).

Давайте небольшой примерчик приведем

In [3]:
# Cгенирируем выборку например из Гамма распределения определенного размера.
# Теперь выполним тест Колмогорова-Смирнова как тест на простую гипотезу, 
# т.е. принадлежит ли эта выборка собственна гамма распределению с параметрами
# генерации
rvs = ss.gamma(1.3, 10, 400).rvs(100,random_state=42)
pvalue = ss.ks_1samp( rvs,ss.gamma.cdf,(1.3, 10, 400) ).pvalue
# p-value = 0.65, а значит мы уверенно принимаем данную гипотезу
print('KS p-value простой гипотезы: ',pvalue)
# А теперь давайте представим, что мы не знаем параметры распределения и сфитируем
# их каким-нибудь методом.

def fit_params(rvs):
    # По умолчанию используется метод максимального правдоподобия
    params_mle = ss.gamma.fit(rvs)
    # Можем использовать метод моментов
    params_mm  = ss.gamma.fit(rvs, method='MM')

    print( 'Параметры с максимизацией правдоподобия (MLE): ',  params_mle)
    print( 'KS p-value MLE: ',  ss.ks_1samp( rvs,ss.gamma.cdf,(params_mle ) ).pvalue )
    print( 'Параметры методом моментов (MM): ',  params_mm)
    print( 'KS p-value MM: ',  ss.ks_1samp( rvs,ss.gamma.cdf,(params_mm ) ).pvalue )

    return params_mle, params_mm

params_mle, params_mm =  fit_params(rvs)

KS p-value простой гипотезы:  0.6494193189265555
Параметры с максимизацией правдоподобия (MLE):  (0.09996545801645901, 26.503932439405734, 3.2761157401038012)
KS p-value MLE:  3.124622953639957e-170
Параметры методом моментов (MM):  (1.268184095938802, 57.89191497820224, 352.5055164803378)
KS p-value MM:  0.9278579917942404


Выше мы вызвали метод fit. В scipy у него есть два параметра MLE - максимизация функции правдоподобия и MM - метод моментов. Обычно MLE дает более точный результат и работает дольше. Но может и совершенно некорректно обучиться на малых выборках.

In [4]:
def plot_pdfs(params_mle,params_mm,N=100):
    x = np.arange(0,1500)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y = ss.gamma(1.3, 10, 400).pdf(x), name = 'Исходное распределение'))
    fig.add_trace(go.Scatter(x=x, y = ss.gamma(*params_mle).pdf(x), name = 'Фитированное MLE'))
    fig.add_trace(go.Scatter(x=x, y = ss.gamma(*params_mm).pdf(x), name = 'Фитированное MM'))

    fig.update_layout( yaxis=dict(range=[0, .005]), title=f'Cравнени pdf, {N} значений в выборке' )

    fig.show()

plot_pdfs(params_mle,params_mm)

С одной стороны у нас MLE дал просто некорректные результаты (кстати говоря, это можно не заметить, если что-то массово тестировать). 
С другой стороны MM обучился на имеющейся выборке из-за чего завысил p-value т.к. сравнивал выборку не с распределеним с исходными параметрами, а с теми, которые лучше подходят к полученной выборке, тем самым максимизируя p-value.

Ради интереса давайте увеличим количество значений в выборке и посмотрим как это все работает.

In [5]:
plot_pdfs( *fit_params( ss.gamma(1.3, 10, 400).rvs(300,random_state=42) ), N=300 )

Параметры с максимизацией правдоподобия (MLE):  (1.3881621547455518, 12.855765652081741, 363.73318967434955)
KS p-value MLE:  0.9941830305807233
Параметры методом моментов (MM):  (1.8238208546586439, -47.59429017931859, 309.99244254300345)
KS p-value MM:  0.8801549950959721


Хорошо видно, что MLE лучше обучился на данных чем MM. И в обоих слчаяз значения p-value завышены.

<h3> Критерий Крамера — Мизеса — Смирнова и Андерсона-Дарлинга </h3>

Строятся похожим образом как и критерий Колмогорова-Смирнова. Так же на вход подаются теоретическая и эмпирическая функции распределения. Только статистика иммет более сложный вид чем супремум модуля разности. Можно посмотреть на статитстики и формулы [тут](https://en.wikipedia.org/wiki/Cram%C3%A9r%E2%80%93von_Mises_criterion) и [тут](https://en.wikipedia.org/wiki/Anderson%E2%80%93Darling_test) . Андерсон-Дарлинг это тот же Мизес, только хвостам функции распределения присвоены большие веса.  

В scipy для них реализованы функции stats.cramervonmises и stats.anderson

Они также не свободны от распределения для сложных гипотез.

Обычно мощности критерия соотносятся как Андерсон-Дарлинг > Мизес > Колмогоров-Смирнов.

<h2> Тесты типа хи2 </h2>

Если предыдущие критерии работали с функцией распределения (cumulative distribution function), то данные функции работают с частотами. Он сравнивает вероятности попадания значения в определенный интервал. Грубо и неправильно говоря - сравнивает теоретическую и эмпирическую гистограммы. И вот чтобы получить эмпирическую гистограмму - нужно сгруппировать данные по бинам, что можно сделать разными способами (в отличии от получения ecdf - эмпирической функции распределения, которая всегда получается только одним споосбом). Поэтому такие тесты называются тестами на группированных данных.

Но для дискретной случайной величины (например для Пуассона) - ничего дополнительно группировать не надо.

Для расчета критерия находится расстояние $ \chi^2 $ как $$\chi^2 = \sum \frac{(O-E)^2}{E}$$
Поделив числитель и знаменатель на $1/N^2$ получим выражение для вероятностей
$$\chi^2 = N\sum \frac{(p_{O}-p_{E})^2}{p_{E}}$$

Эта величина имеет статистику $\chi^2$  c n-1 степенями свободы (n - количество бинов, на которые разбиваются данные). Зная эти две величины мы можем сказать - какова вероятность получить вот эти конкретные данные из конкретного распределения. Т.е. посчитать p-value.

Группировка данных (т.е. формирование бинов) - это отдельная магия, т.к. тут решается классическая задача точности и разброса (по типу принципа неопределенности). Есть способы сделать разбиения минимизируя потери в информации Фишера, но останавливаться мы на этом не будем.

В литературе [говорится](https://www.mathnet.ru/links/ea3ba6ac20c1b2e78f2624de859d6dd1/sjim504.pdf), что при оптимальной группировке такие тесты обладают большей мощностью чем тесты на негруппированных данных. 

<h3> G-тест, Фримен-Тьюки и другие </h3>

Так же по группированным данным придумана еще масса статистик. Например в [G-тесте](https://en.wikipedia.org/wiki/G-test) статистика считается как

$$ G = 2\sum O_i*ln\bigg(\frac{O_i}{E_i}\bigg)$$

Она является обобщением $\chi^2$, из который последний получается разложением логарифма в ряд Тейлора в районе 1, т.е. когда наблюдаемые и теоретические частоты близки. Поэтому $\chi^2$ более чувствителен к выбросам.

Фримена-Тьюки (Freeman-Tukey) статистика выглядит как $$ T^2 =  4*\sum \big( \sqrt{O_i}-\sqrt{E_i} \big)^2 $$ В каких конкретно ситуациях она лучше я не нашел, но наверно где-то предпочтительнее.

Все это обобщается статистикой [Кресси-Рида (Cressie-Read)](https://www.isi-web.org/sites/default/files/import/pdf/123-day3-ips100-the-cressie-read-divergence-st.pdf), где, подставляя различные значения степени, получаем и $\chi^2$ и G и Фримена-Тьюки и другие статистики. В scipy для него есть функция scipy.stats.power_divergence, где параметром lambda_ можно выбрать один из 6ти стат тестов. Собственное если залезть внутрь сайпи $\chi^2$ , то мы там вызов power_divergence и увидим.

Давайте возьмем нашу выборку и потестируем ее как простую и сложные гипотезы. Просто чтобы примерно представлять, как оно выглядит. В идеале значения p-value в обоих случаях должны быть близки

In [7]:
def get_pvalues_grupp(dist, params: list, lenght: int, f_obs: np.array):

    pr = dist.cdf(bins,*params)
    probabilites = pr[1:] - pr[:-1]
    f_exp = probabilites * lenght / probabilites.sum()

    for lambda_ in ('pearson','log-likelihood','cressie-read'):
        pvalue = ss.power_divergence(f_obs,f_exp,lambda_=lambda_).pvalue
        print(lambda_+' pvalue: {:.4f}'.format(pvalue) )

    return f_exp


rvs = ss.gamma(1.3, 10, 400).rvs(300,random_state=42)

hist, bins = np.histogram(rvs,bins='auto',density=True)
x_pdf = np.linspace(0,max(rvs),500)
f_obs, bins = np.histogram(rvs,bins='auto')

params_mle, params_mm = fit_params( rvs )

print('\n===Тестируем простую гипотезу===')
get_pvalues_grupp(ss.gamma,(1.3, 10, 400), len(rvs), f_obs);
print('\n===Тестируем сложную гипотезу===')
f_exp = get_pvalues_grupp(ss.gamma,params_mle, len(rvs), f_obs)

''' Рисуем гистограмму и расчитанные pdf '''

fig = make_subplots(rows=1, cols=2
,subplot_titles=("ePDF и PDF", "Ожидаемые и наблюдаемые частоты"))
fig.add_trace(go.Scatter(x=x_pdf, y=ss.gamma(1.3, 10, 400).pdf(x_pdf)
                                                , name='Pdf'), row=1, col=1)
fig.add_trace(go.Bar(x=bins, y=hist, name='Histplot'), row=1, col=1)

fig.add_trace(go.Bar(x=bins, y=f_exp, name='Expected freq'), row=1, col=2)
fig.add_trace(go.Bar(x=bins, y=f_obs, name='Observed freq'), row=1, col=2)
fig.show()



Параметры с максимизацией правдоподобия (MLE):  (1.3881621547455518, 12.855765652081741, 363.73318967434955)
KS p-value MLE:  0.9941830305807233
Параметры методом моментов (MM):  (1.8238208546586439, -47.59429017931859, 309.99244254300345)
KS p-value MM:  0.8801549950959721

===Тестируем простую гипотезу===
pearson pvalue: 0.5358
log-likelihood pvalue: 0.2460
cressie-read pvalue: 0.4672

===Тестируем сложную гипотезу===
pearson pvalue: 0.4786
log-likelihood pvalue: 0.2600
cressie-read pvalue: 0.4334


<h2> Монте-Карло </h2>

Хорошо, если все эти критерии нужно тестировать и какой они обладают мощностью, то может просто как-то все считать численно?

Кажется мы можем вычислить выборочные параметры распределения (например среднее и дисперсию для нормального распределения). Потом нагенерировать выборок той же длины сколько дуще угодно. И посчитать на них нужные статистики, например Колмогорова-Смирнова. Давайте сделаем это. 

In [ ]:
def ks_stat(data
            ,cdf
            ,params):

    x = np.sort(data)
    cdfval = cdf(x,*params)
    n = len(x)
    Dpls = max( np.arange(1.,n+1)/n - cdfval )
    Dmin = max( cdfval - np.arange(0.,n)/n )

    return max([Dmin,Dpls])

def get_ecdf(t):
    # Возвращает эмпирическую функцию распределения и значения переменной
    # x, ecdf
    x, cnt = np.unique(t, return_counts=True)
    cumsum = np.cumsum( cnt )
    return x, cumsum/cumsum[-1]

def bad_monte_carlo(dist
                    ,data
                    ,params
                    ,size=999):

    S = np.zeros(size)
    lendata = len(data)
    # Посчитали статистику между исходными данными и сфитированным распределением 
    S_obs = ks_stat(data,dist.cdf,params)

    for i in range(size):
        rvs = dist.rvs(*params,size=lendata)
        S[i] = ks_stat(rvs,dist.cdf,params)

    print('Monte-Carlo P-value: {:.3f}'.format( (S >= S_obs).sum()/size ) )

def _anderson_darling(dist, data):
    x = np.sort(data, axis=-1)
    n = data.shape[-1]
    i = np.arange(1, n+1)
    Si = (2*i - 1)/n * (dist.logcdf(x) + dist.logsf(x[..., ::-1]))
    S = np.sum(Si, axis=-1)
    return -n - S

def _cramer_von_mises(dist, data):
    x = np.sort(data, axis=-1)
    n = data.shape[-1]
    cdfvals = dist.cdf(x)
    u = (2*np.arange(1, n+1) - 1)/(2*n)
    w = 1 / (12*n) + np.sum((u - cdfvals)**2, axis=-1)
    return w

In [ ]:
# Давайте проверим простую гипотезу
data = ss.gamma(1.3, 10, 400).rvs(100,random_state=42)
print('===Простая гипотеза===')
bad_monte_carlo(ss.gamma
                ,data
                ,(1.3, 10, 400))

kstestp = ss.ks_1samp( data 
                      ,ss.gamma.cdf
                      ,(1.3, 10, 400) ).pvalue

print('KS test: {:.3f}'.format( kstestp ) )  

# Ну и сложную, которую раньше тестировали
params_mm  = ss.gamma.fit(data, method='MM')

print('\n===Сложная гипотеза===')
bad_monte_carlo(ss.gamma
                ,data
                ,params_mm )

kstestp = ss.ks_1samp(data 
                      ,ss.gamma.cdf
                      ,params_mm ).pvalue

print('KS test: {:.3f}'.format( kstestp ) ) 

===Простая гипотеза===
Monte-Carlo P-value: 0.643
KS test: 0.649

===Сложная гипотеза===
Monte-Carlo P-value: 0.926
KS test: 0.928


Получается нет никакого резона в Монте-Карло? Те же числа, только считать дольше?

Но мы неправильно его реализовали и сравнивали каждый раз с тем же самым распределеним. А как мы видели выше - это завышает p-value. Можно применить следующий ход. 
1. У нас есть экспериментальная выборка - data. По ней мы находит вектор параметров нашего распределения params.
2. Далее на каждом шаге мы генерируем rvs - случайную выборку того же размера (все как и раньше пока)
3. Но вот теперь мы находим _params обучаясь на этой новой выборке.
4. Находим статистику T(_params,rvs)

Т.е. мы смотрим, как себя будут вести статистики (напомню - это функция от данных/способа обучения) на сгенерированных выборках.

Давайте посмотрим на наш пример

In [ ]:
def good_monte_carlo(dist
                    ,data
                    ,params
                    ,size=999
                    ,method='MLE'
                    ,statistic='ks'
                    ,print_pvalue=False):
    '''dit - объект scipy stats.name_of_dist, data - сгенерированная выборка,
    params - параметры распределения (истинные - тогда тестируется простая
    гипотеза, посчитанные на выборке - тогда сложная)
    size - количество симуояций Монте-Карло  '''
    S = np.zeros(size)
    lendata = len(data)
    # Посчитали статистику между исходными данными и сфитированным распределением 
    if statistic == 'ks':
        S_obs = ks_stat(data,dist.cdf,params)
    elif statistic == 'ad':
        S_obs = _anderson_darling(dist(*params),data)
    elif statistic == 'cvm':
        S_obs = _cramer_von_mises(dist(*params),data)

    for i in range(size):
        # Сгенерировали случайную выборку исходного размера
        rvs = dist.rvs(*params,size=lendata)
        # Нашли параметры распределения 
        _params = dist.fit(rvs,method=method)
        # Посчитали статистику
        if statistic == 'ks':
            S[i] = ks_stat(rvs,dist.cdf,_params)
        elif statistic == 'ad':
            S[i] = _anderson_darling(dist(*_params),rvs)
        elif statistic == 'cvm':
            S[i] = _cramer_von_mises(dist(*_params),rvs)

    pvalue =  (S >= S_obs).sum() / size
    if print_pvalue:
        print('Monte-Carlo P-value: {:.3f}'.format( pvalue ) )
    return pvalue

In [ ]:
# Давайте проверим простую гипотезу
data = ss.gamma(1.3, 10, 400).rvs(100,random_state=42)
print('===Простая гипотеза===')
good_monte_carlo(ss.gamma
                ,data
                ,(1.3, 10, 400)
                ,size=9999
                ,method='MM'
                ,statistic='ks'
                ,print_pvalue=True)
kstestp = ss.ks_1samp( data ,ss.gamma.cdf,(1.3, 10, 400) ).pvalue
print('KS test: {:.3f}'.format( kstestp ) )  

# Ну и сложную, которую раньше тестировали
params_mm  = ss.gamma.fit(data, method='MM')
print('\n===Сложная гипотеза===')
good_monte_carlo(ss.gamma
                ,data
                ,params_mm
                ,method='MM'
                ,size=9999
                ,statistic='ks'
                ,print_pvalue=True)
kstestp = ss.ks_1samp(data ,ss.gamma.cdf,params_mm ).pvalue
print('KS test: {:.3f}'.format( kstestp ) ) 

===Простая гипотеза===
Monte-Carlo P-value: 0.522
KS test: 0.649

===Сложная гипотеза===
Monte-Carlo P-value: 0.879
KS test: 0.928


P-value уменьшилось, хотя это и не поражает воображение. К слову этот метод реализован в scipy функцией stats.goodness_of_fit, только на момент написания статьи использование метода моментов вызывало пробелмы.



<h1> Какое у нас все таки распределение? </h1>

Теперь давайте попробуем сравнить некоторые наши критерии. Распределение выше примерно соответствует распределению среднегов времени диалога в одном из интервалов. И вот вопрос - это гамма распределение или, например, логнормальное?

Давайте посмотрим, на сколько легко наши критерии отличат одно от другого. Сгенерируем несколько выборок и посмотрим на распределение p-value. По умолчанию 
в scipy используется MLE метод.

In [70]:
import plotly.express as px

def statsign(basedist
            ,distname1:str
            ,distname2:str
            ,method1='MLE'
            ,method2='MLE'
            ,N=100
            ,l=300
            ,testtype='simple_ks'
            ,mkparams={} ) -> pd.DataFrame:
    ''' basedist - базовое распределение, из которого генерируется выборка
    distname1 - преове распределение для сравненияm, method1 - метод фитинга 1
    distname2 - второе распределение для сравнения, method2 - метод фитинга 2
    N - количество элементов в выборке
    l - количество выборок
    testtype: simple_ks - простой К-С тест, mk - монте-карло
    mkparams - монте-карло словарь
    '''
    results = np.zeros([l,2])
    dist1 = getattr(ss,distname1)
    dist2 = getattr(ss,distname2)

    for i in tqdm( range(l) ):
        data = basedist.rvs(N)

        params1 = dist1.fit(data, method=method1)
        params2 = dist2.fit(data, method=method2)

        if testtype == 'simple_ks':
            results[i,0] = ss.ks_1samp( data ,dist1.cdf,params1 ).pvalue
            results[i,1] = ss.ks_1samp( data ,dist2.cdf,params2 ).pvalue
        elif testtype == 'mk':
            results[i,0] = good_monte_carlo(dist1,data,params1,method=method1,**mkparams)
            results[i,1] = good_monte_carlo(dist2,data,params2,method=method2,**mkparams)

    return pd.DataFrame({distname1:results[:,0],distname2:results[:,1]})

def plot_hist(pvalues:pd.DataFrame) -> None:

    fig = px.histogram(
        pvalues,
        title='Распределение p-value',
        histnorm='probability',
        barmode="overlay",
        nbins = 20 )
    fig.show()

In [58]:
# Сохраним в словарике разные p-value для разных тестов
pvalues_tests = {}

In [103]:
pvalues = statsign(ss.gamma(1.3, 10, 400)
                   ,'gamma'
                   ,'lognorm'
                   ,N=100
                   ,l=300)

pvalues = pvalues[pvalues.gamma >0.01] # Исключили неправильный фит
pvalues_tests['ks'] = pvalues
delta = pvalues.gamma - pvalues.lognorm > 0
print( delta.value_counts()/len(delta)  )

plot_hist(pvalues)

100%|██████████| 300/300 [00:02<00:00, 104.50it/s]

True     0.560166
False    0.439834
Name: count, dtype: float64


Выше мы сгенерировали 300 выборок из Гамма распределения по 100 элементов. Далее мы посмотрели - в каком количестве **пар** p-value Гамма распределения было больше чем p-value логнормального (значение True). При этом пары, где p-value гамма распределения было меньше 0.01 мы выкинули, т.к. это говорит о некорректной работе MLE оптимизатора.

Получили примерно одинаковое количество ложных и истинных срабатываний. Причем в большом проценте экспериментов p-value логнормального распределения было в районе 0.9, см. гистограмму.

Теперь давайте используем Монте-Карло симуляцию в тех же условиях.

In [104]:
pvalues = statsign(ss.gamma(1.3, 10, 400)
                   ,'gamma'
                   ,'lognorm'
                   ,N=100
                   ,testtype='mk'
                   ,l=300)

pvalues = pvalues[pvalues.gamma > 0.01]
pvalues_tests['monte-karlo'] = pvalues 

delta = pvalues.gamma - pvalues.lognorm > 0
print( delta.value_counts()/len(delta)  )
plot_hist(pvalues)

  0%|          | 0/300 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.12/site-packages/scipy/stats/_continuous_distns.py:6545: RuntimeWarning:

invalid value encountered in log

100%|██████████| 300/300 [1:05:18<00:00, 13.06s/it]

True     0.691057
False    0.308943
Name: count, dtype: float64


И тут алгоритм ошибается куда в меньшем количестве случаев в тех-же условиях (70 на 30) при том же критерии. Понятно, что строго говоря нужно получить p-value на данное p-value и если мы еще раз запустим процедуру, то результат будет несколько отличатся. Но общая картина ясна. 

Заметим, что с ростом выборки все становится заметно лучше и уже при N=600 мы в 95% случаев мы не ошибемся, если выберем распределение с большим p-value (ниже представлены применение простого Колмогорова, без Монте-Карло)

In [99]:
pvalues = statsign(ss.gamma(1.3, 10, 400)
                   ,'gamma'
                   ,'lognorm'
                   ,N=600
                   ,l=10000)

pvalues = pvalues[pvalues.gamma >0.01] # Исключили неправильный фит
delta = pvalues.gamma - pvalues.lognorm > 0
print( delta.value_counts()/len(delta)  )

plot_hist(pvalues)

100%|██████████| 10000/10000 [01:25<00:00, 117.29it/s]


True     0.944528
False    0.055472
Name: count, dtype: float64


Тем самым, если у нас вот прям нет способа увеличить выборку - то нужно использовать правильное Монте-Карло, посмотреть на разные статистики и выбрать ту, которая делает критерий более мощным. 

<h1> Выводы </h1>

1. Нет единого аналитического подхода для всех объемов выборок и всех распределений. Самый очевидный вариант "из коробки" может легко обмануть.
2. Для каждой конкретной задачи лучше провести моделирование - на сколько мощны ваши критерии в ней. Какой лучше выбрать критерий и какой объем выборки необходим. Надеюсь материал выше в этом поможет.
3. Я тут на этом не останавливался, но существуют специализированные тесты для сложных гипотез (например тест Лиллиефорса для нормального распределения, который собственно адаптированный Колмогоров-Смирнов), если есть возможность - лучше пользоваться ими. 